## Import Libraries

In [ ]:
import os
import re
import pathlib
import json
import sqlalchemy as db

from datetime import datetime, date

import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpf
from statsmodels.graphics.gofplots import qqplot

import plotly.offline as pyo
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)


## Configurations

In [ ]:
import quandl
import pandas_datareader as pdr
from pycoingecko import CoinGeckoAPI
import yfinance as yf
import twitter

# Quandl API
quandl_key = "f6Y8avFQZwXp37ftC1_6"
quandl.ApiConfig.api_key = quandl_key
cg = CoinGeckoAPI()

# EIA API
eia_key = "320c16a416c0cb06ed1a737576ec3ee0"
eia_id = ""
eia_url = f"http://api.eia.gov/series/?api_key={eia_key}&series_id={eia_id}"

# Twitter API
tw_consumer_key = ""
tw_consumer_secret = ""
tw_token = ""
tw_token_secret = ""

#api = twitter.Api(consumer_key=tw_consumer_key,
#                 consumer_secret=tw_consumer_secret,
#                  access_token_key=tw_token,
#                  access_token_secret=tw_token_secret)

twitter_search_url = 'https://twitter.com/search?q=(bitcoin%20OR%20ethereum%20OR%20cryptocurrency)&src=typed_query'
#results = api.GetSearch(twitter_search_url)

project_dir = pathlib.Path().absolute()
workdir = project_dir.parent
# print(f"{project_dir}\n{workdir}")

## Build functions

#### Local Functions for Data manipulation

In [ ]:
def check_items(list1, list2):
    check = all(item in list1 for item in list2)
    return check

def value_mapper(df, col_name_to_map, new_col_name, mapping_dict):
    df[new_col_name] = df[col_name_to_map].str.upper().map(mapping_dict)
    return df

def df_subset(df, df_name, mapping_dict):
    columns_mapped = [i for i in mapping_dict[df_name][0]]
    df_subset = df[[column for column in df.columns if column in columns_mapped]]
    return df_subset


def column_rename(df, df_name, mapping_dict):
    df_renamed = df.rename(columns=mapping_dict[df_name][0])
    return df_renamed


def df_date_to_col(df):
    df_new_index = df.reset_index()
    df_date_col = df_new_index.rename(columns={'index': 'Date', 'date': 'Date'})
    return df_date_col


def sql_preprocess(df, df_name, mapping_dict):
    df_sub = df_subset(df, df_name, mapping_dict)
    df_ren = column_rename(df_sub, df_name, mapping_dict)
    df_prep = df_date_to_col(df_ren)
    return df_prep

def load_to_sql(table_name):
    engine = db.create_engine('sqlite:///crypto.db', echo=True)
    meta = db.MetaData(engine)

    with engine.connect() as con:
        df_btc_prep.to_sql(table_name, con=con, if_exists='replace', index=False)
        con.close()

def create_date_table(start='1900-01-01', end='2099-12-31'):
   df = pd.DataFrame({"Date": pd.date_range(start, end)})
   df["Week_day"] = df.Date.dt.day_name()
   df["Day"] = df.Date.dt.day
   df["Month"] = df.Date.dt.month
   df["Week"] = df.Date.dt.isocalendar().week
   df["Quarter"] = df.Date.dt.quarter
   df["Year"] = df.Date.dt.year
   df.insert(0, 'DateID', (df.Year.astype(str) + df.Month.astype(str).str.zfill(2) + df.Day.astype(str).str.zfill(2)).astype(int))
   return df

def calculate_average(row_col):
    return row_col.mean()


#### Data Import functions

In [ ]:
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def get_json_data(data_id, json_path, provider):
    """Download and cache JSON data, return as a dataframe."""
    
    cache_path = f"{workdir}\\02_Resources\{data_id}.json"
    try:        
        f = open(cache_path, 'rb')
        df = pd.read_json(f)   
        print(f"Loaded {cache_path} from cache")
    except (OSError, IOError) as e:
        print(f"Downloading {data_id} from {provider}")
        df = pd.read_json(json_path)
        df.to_json(cache_path)
        print(f"Cached {data_id} at {cache_path}")
    return df



## Quandl Data import
# Bitcoin Charts Exchange Rate Data

def get_quandl_data(quandl_id):
    """" Download quandl data and load to json """
    
    data_id = quandl_id.replace('/', '-')
    provider = "Quandl"
    df = quandl.get(quandl_id, returns="pandas")
    json_path = df.to_json()
    return get_json_data(data_id, json_path, provider)


    
## Yahoo Finance Data import

def get_yfinance_data(ticker):
    """" Download yahoo finance data and load to json """
    
    data_id = re.sub(r'\W+', '_', ticker)
    provider = "Yahoo"
    df = yf.download(ticker)
    json_path = df.to_json()
    return get_json_data(data_id, json_path, provider)

## Data Reader Data import

def get_datareader_data(datareader_id, provider):
    """" Download datareader data and load to json """
    data_id = datareader_id
    df = pdr.DataReader(datareader_id, provider)
    if df.shape[0]!=df.index.nunique():
        idx = np.unique(df.index.values, return_index=True)[1]
        df = df.iloc[idx]    
            
    json_path = df.to_json()
    return get_json_data(data_id, json_path, provider)


## Poloniex Data import

base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2014-01-01', '%Y-%m-%d') # get data from the start of 2014
end_date = datetime.now() # up until today
period = 86400 # pull daily data (86,400 seconds per day)

def get_poloniex_data(poloniex_pair):
    """Retrieve cryptocurrency data from poloniex"""
    
    data_id = poloniex_pair
    provider="Poloniex"
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), period)
    df = get_json_data(data_id, json_url, poloniex_pair)
    df = df.set_index('date')
    return df


## Data Collection

### Collect Market & Exchange data

In [ ]:
# Market (Asset) Data from Coingecko.com

markets_list = cg.get_coins_markets(vs_currency='usd')
market_names = []
for item in markets_list:
    market_names.append(item['name'])
df_cg_markets = pd.DataFrame.from_dict(markets_list)
df_cg_markets.columns = df_cg_markets.columns.str.title()
df_cg_markets.head()

In [ ]:
# Exchange Data from Coingecko.com

exchanges_list = cg.get_exchanges_list()
exchange_names = []
for item in exchanges_list:
    exchange_names.append(item['name'])
df_cg_exchanges = pd.DataFrame.from_dict(exchanges_list)
df_cg_exchanges.columns = df_cg_exchanges.columns.str.title() 
df_cg_exchanges.head()

In [ ]:
# jprint(markets_list)

In [ ]:
# jprint(exchanges_list)

In [ ]:
print(f"No. of Assests:   {len(market_names)}\n" \
      f"No. of Exchanges: {len(exchange_names)}")

In [ ]:
print(df_cg_markets['Symbol'])

In [ ]:
print(df_cg_exchanges['Id'])

### Collect Bitcoin Data from different exchanges from Quandl

In [ ]:
from quandl.errors.quandl_error import NotFoundError

exchanges = df_cg_exchanges['Id'].str.upper()

df_run_all = []
    
for exchange in exchanges:
    exchange_code = f"BCHARTS/{exchange}USD"
   
    try:
        df = get_quandl_data(exchange_code)
        df['Exchange'] = exchange
        df_run_all.append(df)
        
    except NotFoundError as e:
        continue

df_quandl_btc_data = pd.concat(df_run_all)

In [ ]:
df_quandl_btc_data.rename(columns={'Volume (BTC)': 'Volume_BTC', 'Volume (Currency)': 'Volume_USD'}, inplace=True)
df_quandl_btc_data.sort_index(inplace=True)
df_quandl_btc_data.tail()

### Collect Bitcoin Data from Poloniex

In [ ]:
df_poloniex_btc_data = get_poloniex_data("USDT_BTC")
df_poloniex_btc_data.columns= df_poloniex_btc_data.columns.str.title()
df_poloniex_btc_data = df_poloniex_btc_data.rename(columns={'Volume': 'Volume_USD', 'Quotevolume': 'Volume_BTC',  
                                                     'Weightedaverage': 'Weighted Price'})

df_poloniex_btc_data['Exchange'] = 'POLONIEX'
column_names_btc=['Open', 'High', 'Low', 'Close', 'Volume_BTC', 'Volume_USD', 'Weighted Price', 'Exchange']
df_poloniex_btc_data = df_poloniex_btc_data.reindex(columns=column_names_btc)
df_poloniex_btc_data.sort_index(inplace=True)
df_poloniex_btc_data.tail()

### Append Poloniex Data to have the consoildated BTC Dataset

In [ ]:
df_btc_data = df_quandl_btc_data.append(df_poloniex_btc_data)
df_btc_data.sort_values(by=['Exchange'])
df_btc_data['Market'] = "BTC"
df_btc_data = df_btc_data.sort_index()
df_btc_data.tail()

## Data Ingestion

### Collect Altcoin Data from Quandl

In [ ]:
from quandl.errors.quandl_error import NotFoundError

markets = df_cg_markets['Symbol'].str.upper()

df_run_mkt_new = []
    
for market in markets:
    market_code = f"BITFINEX/{market}BTC"
   
    try:
        df = get_quandl_data(market_code)
        df['Market'] = market
        df_run_mkt_new.append(df)
        
    except NotFoundError as e:
        continue

df_altcoin_quandl = pd.concat(df_run_mkt_new)

In [ ]:
df_altcoin_quandl_data = df_altcoin_quandl.sort_index()
df_altcoin_quandl_data.tail()

### Collect Altcoin data from Poloniex 

In [ ]:
altcoins = markets
df_run_altcoin_new = []

for altcoin in altcoins:
    coinpair = f"BTC_{altcoin}"
    try:
        df = get_poloniex_data(coinpair)
        df['Market'] = altcoin
        df_run_altcoin_new.append(df)
                
    except ValueError as e:
        continue

df_altcoin_poloniex = pd.concat(df_run_altcoin_new)

In [ ]:
df_altcoin_poloniex_data = df_altcoin_poloniex.sort_index()
df_altcoin_poloniex_data.tail()

In [ ]:
print(f"Number of observations Quandl Dataset  : {df_altcoin_quandl_data.shape[0]}\n" \
      f"Number of observations Poloniex Dataset: {df_altcoin_poloniex_data.shape[0]}\n")

print(f"Number of Assets quoted Quandl   : {len(set(df_altcoin_quandl_data['Market']))}\n" \
      f"Number of Assets quoted Poloniex : {len(set(df_altcoin_poloniex_data['Market']))}")

In [ ]:
print("Sum of N/As Quandl Dataset:", df_altcoin_quandl_data.isna().sum().sum())
print("Sum of N/As Poloniex Dataset:", df_altcoin_poloniex_data.isna().sum().sum())

#### Take Poloniex for Altcoin data

In [ ]:
df_altcoin_poloniex_data.columns = df_altcoin_poloniex_data.columns.str.title()
df_altcoin_poloniex_data = df_altcoin_poloniex_data.rename(columns={'Volume': 'Volume_BTC', 'Quotevolume': 'Volume_Market',  
                                                     'Weightedaverage': 'Weighted Price'})

df_altcoin_poloniex_data['Exchange'] = 'POLONIEX'
column_names_altcoin=['Open', 'High', 'Low', 'Close', 'Volume_BTC', 'Volume_Market', 'Weighted Price', 'Exchange', 'Market']
df_altcoin_poloniex_data = df_altcoin_poloniex_data.reindex(columns=column_names_altcoin)

df_altcoin_poloniex_data.tail()

### Inject Other relevant Data

In [ ]:
# Get Bitcoin mining data from Quandl / Bitcoinwatch

df_mining_data = get_quandl_data('BITCOINWATCH/MINING')
df_mining_data['Market'] = 'BTC'
df_mining_data['Source'] = 'BITCOINWATCH'
df_mining_data.tail()

In [ ]:
# Get Commodities, Indices, etc. from Yahoo Finance

tickers = {
    'Oil_Price': 'CL=F',
    'Gold_Price': 'GC=F',
    'SP500': '^GSPC',
    'DJI': '^DJI',
    'Nasdaq': '^IXIC',
    'Tesla': 'TSLA'
}

asset_list = []

for name, ticker in tickers.items():
    df = get_yfinance_data(ticker)
    df['Asset'] = name
    df['Symbol'] = re.sub(r'\W+', '', ticker)
    df['Source'] = 'YAHOO'
    asset_list.append(df)
    
df_asset_data = pd.concat(asset_list)
df_asset_data.tail()

In [ ]:
tickers = {
    'SP500': '^GSPC',
    'DJI': '^DJI',
    'Nasdaq': '^IXIC',
    'Tesla': 'TSLA'
}

asset_list = []

for name, ticker in tickers.items():
    df = get_yfinance_data(ticker)
    df['Asset'] = name
    df['Symbol'] = re.sub(r'\W+', '', ticker)
    df['Source'] = 'YAHOO'
    asset_list.append(df)
    
df_asset_data = pd.concat(asset_list)
df_asset_data.tail()

In [ ]:
asset_list

In [ ]:
# Collect Investor sentiment Data from Quandl

aaii = get_quandl_data('AAII/AAII_SENTIMENT')
inv_sentiment = aaii[['Bullish', 'Bearish', 'Bullish 8-Week Mov Avg', 'Bull-Bear Spread']]
inv_sentiment.rename(columns={'Bullish 8-Week Mov Avg':'Bull_MA_8wk'}, inplace=True)
inv_sentiment['Bear_MA_8wk'] = inv_sentiment['Bearish'].rolling(window=8).mean()
inv_sentiment.tail()

In [ ]:
# Upsample the data to Daily from Weekly

upsampled = inv_sentiment.resample('D')
df_inv_sentiment = upsampled.interpolate(method='linear')
df_inv_sentiment.tail()

In [ ]:
# EIA Data

eia_id = "STEO.BREPUUS.M" # Brent Crude oil prices monthly
eia_url


## Data Organization

### Create Date Table

In [ ]:
# Create Calendar Table
df_dates = create_date_table()
df_dates.set_index('Date', inplace = True)
df_dates.tail()

### Preprocessing

<p><strong>Relevant Dataframes:</strong></p>
<ul style="line-height:180%">
    <li>BTC Data: df_btc_data</li>
    <li>Altcoin Data: df_altcoin_poloniex_data</li>
    <li>Fin Assets Data: df_asset_data</li>
    <li>Mining Data: df_mining_data</li>
    <li>Exchanges Data: df_cg_exchanges</li>
    <li>Markets Data: df_cg_markets</li>
    <li>Coingecko BTC Price Data: df_cg_btc_data</li>
    <li>Yahoo BTC Price Data: df_yahoo_btc_data</li>
    <li>Date Table: df_dates</li>
</ul>

In [ ]:
# Create copies of the imported data that is being transformed

df_btc = df_btc_data.copy()
df_altcoin = df_altcoin_poloniex_data.copy()
df_fin_asset = df_asset_data.copy()
df_mining = df_mining_data.copy()
df_exchanges = df_cg_exchanges.copy()
df_markets = df_cg_markets.copy()


# Create dictionary of the copied dataframes
data_frames = {
    'BTC Data': df_btc,
    'Altcoin Data': df_altcoin,
    'Fin Assets Data': df_fin_asset,
    'Mining Data': df_mining,
    'Exchanges Data': df_exchanges,
    'Markets Data': df_markets,
    'Date Table': df_dates
}

# Organize the Dataframe column names into a dictionary
data_frames_col_names = {}
for df_name, df in data_frames.items():
    data_frames_col_names[df_name] = df.columns
data_frames_col_names

In [ ]:
# Add the Fin Assets Exchange Data to Exchange List 

exchanges_list_cg = pd.unique(df_exchanges['Id'].str.upper())
exchanges_list_cg= np.append(exchanges_list_cg, ['YAHOO'])
exchanges_list_cg.sort()
exchanges_list_cg

print(f"{exchanges_list_cg}")

In [ ]:
# Add the Fin Assets Market data to Market List

markets_list_cg = pd.unique(df_markets['Symbol'].str.upper())
add_markets = np.unique(df_fin_asset['Symbol'])
markets_list_cg = np.append(markets_list_cg, add_markets)
markets_list_cg.sort()
markets_list_cg

print(f"{markets_list_cg}")

In [ ]:
# Create ID Columns for Exchanges

exchange_mapping = {}
for exchange_id, exchange_value in enumerate(exchanges_list_cg):
    exchange_mapping.update({exchange_value:exchange_id+1})
    
value_mapper(df_exchanges, 'Id', 'ExchangeID', exchange_mapping)
df_exchanges[['Id', 'Name', 'ExchangeID']].sort_values(by='Name').tail()

In [ ]:
# Create ID Columns for Markets

market_mapping = {}
for market_id, market_value in enumerate(markets_list_cg):
    market_mapping.update({market_value:market_id+1})

value_mapper(df_markets, 'Symbol', 'MarketID', market_mapping)
df_markets[['Id', 'Symbol', 'MarketID']].sort_values(by='Id').tail()

In [ ]:
# Map the Exchange and Market IDs to the BTC, Altcoin and Fin Asset Data

value_mapper(df_btc, 'Exchange', 'ExchangeID', exchange_mapping)
value_mapper(df_btc, 'Market', 'MarketID', market_mapping)

value_mapper(df_altcoin, 'Exchange', 'ExchangeID', exchange_mapping)
value_mapper(df_altcoin, 'Market', 'MarketID', market_mapping)

value_mapper(df_fin_asset, 'Source', 'ExchangeID', exchange_mapping)
value_mapper(df_fin_asset, 'Symbol', 'MarketID', market_mapping)

value_mapper(df_mining, 'Market', 'MarketID', market_mapping)

print(df_btc.head())
print(df_altcoin.head())
print(df_fin_asset.head())
print(df_mining.head())

##### Create Mapping for subsetting the Data as a preparation to load into the DB

In [1]:
data_frames_col_translate = {
    'BTC Data': [
        {
            'Open': 'Open', 
            'High': 'High', 
            'Low': 'Low', 
            'Close': 'Close', 
            'Volume_BTC': 'Volume_BTC', 
            'Volume_USD': 'Volume_USD',
            'Weighted Price': 'WeightedPrice', 
            'MarketID': 'MarketID',
            'ExchangeID': 'ExchangeID', 
            'Date': 'Date'
        }],
    'Altcoin Data': [
        {
            'Open': 'Open', 
            'High': 'High', 
            'Low': 'Low', 
            'Close': 'Close', 
            'Volume_BTC': 'Volume_BTC',
            'Volume_Market': 'Volume_Market', 
            'Weighted Price': 'WeightedPrice', 
            'MarketID': 'MarketID',
            'ExchangeID': 'ExchangeID', 
            'Date': 'Date'
        }],
    'Fin Assets Data': [
        {
            'Open': 'Open', 
            'High': 'High', 
            'Low': 'Low', 
            'Close': 'Close', 
            'Adj Close': 'AdjClose', 
            'Volume': 'Volume', 
            'Asset': 'Asset',
            'Symbol': 'Symbol',
            'Source': 'Source',
            'MarketID': 'MarketID',
            'ExchangeID': 'ExchangeID',
            'Date': 'Date'
        }],
    'Mining Data': [
        {
            'Total BTC': 'Total_BTC', 
            'Market Cap': 'MarketCap', 
            'Transactions last 24h': 'TR_24h',
            'Transactions avg. per hour': 'TR_Avg_h', 
            'Bitcoins sent last 24h': 'BTC_Sent_24h',
            'Bitcoins sent avg. per hour': 'BTC_Sent_Avg_h', 
            'Count': 'BTC_Count', 
            'Blocks last 24h': 'Blocks_24h',
            'Blocks avg. per hour': 'Blocks_Avg_h', 
            'Difficulty': 'Difficulty', 
            'Next Difficulty': 'Next_Difficulty',
            'Network Hashrate Terahashs': 'Hashrate_Tera', 
            'Network Hashrate PetaFLOPS': 'Hashrate_Peta',
            'MarketID': 'MarketID',
            'Date': 'Date'
        }],
    'Exchanges Data': [
        {
            'ExchangeID': 'ExchangeID', 
            'Id': 'Exchange',
            'Name': 'ExchangeName', 
            'Year_Established': 'Year_Est', 
            'Country': 'Country', 
            'Has_Trading_Incentive': 'Trading_Inc',
            'Trust_Score': 'Trust_Score',
            'Trust_Score_rank': 'Rank_Trust',
            'Trade_Volume_24h_Btc': 'Trade_Vol_24h',
            'Trade_Volume_24h_Btc_Normalized': 'Norm_Trade_Vol'
        }],
    'Markets Data': [
        {
            'MarketID': 'MarketID',
            'Name': 'Market',
            'Symbol': 'Symbol',
            'Id': 'MarketName',
            'Market_Cap': 'MarketCap',
            'Market_Cap_Rank': 'Rank_MarketCap',
            'Total_Volume': 'Total_Volume',
            'Total_Supply': 'Total_Supply',
            'Max_Supply': 'Max_Supply'
        }],
    'Date Table': [
        {
            'Date': 'Date', 
            'Week_day': 'Week_day', 
            'Day': 'Day', 
            'Month': 'Month', 
            'Week': 'Week', 
            'Quarter': 'Quarter',
            'Year': 'Year'
        }]
}

In [2]:
columns_mapped = [i for i in data_frames_col_translate['BTC Data'][0]]

In [5]:
data_frames_col_translate['BTC Data'][0]

{'Open': 'Open',
 'High': 'High',
 'Low': 'Low',
 'Close': 'Close',
 'Volume_BTC': 'Volume_BTC',
 'Volume_USD': 'Volume_USD',
 'Weighted Price': 'WeightedPrice',
 'MarketID': 'MarketID',
 'ExchangeID': 'ExchangeID',
 'Date': 'Date'}

### Preprocessed Data

In [ ]:
# Preprocess Data for SQL DB load

df_btc_prep = sql_preprocess(df_btc, 'BTC Data', data_frames_col_translate)
df_altcoin_prep = sql_preprocess(df_altcoin, 'Altcoin Data', data_frames_col_translate)
df_fin_asset_prep = sql_preprocess(df_fin_asset, 'Fin Assets Data', data_frames_col_translate)
df_mining_prep = sql_preprocess(df_mining, 'Mining Data', data_frames_col_translate)
df_exchanges_prep = sql_preprocess(df_exchanges, 'Exchanges Data', data_frames_col_translate)
df_markets_prep = sql_preprocess(df_markets, 'Markets Data', data_frames_col_translate)
df_dates_prep = sql_preprocess(df_dates, 'Date Table', data_frames_col_translate)

# Additional manual preprocessing

df_markets_prep = df_markets_prep.drop('Date', axis=1)
df_exchanges_prep = df_exchanges_prep.drop('Date', axis=1)
df_exchanges_prep['Trading_Inc'] = df_exchanges_prep['Trading_Inc'].replace({True: 1, False: 0})


In [ ]:
df_exchanges_prep

In [ ]:
# Saving the data into csv Files for external DB load

df_btc_prep.to_csv('btc_dataset_for_DB.csv')
df_altcoin_prep.to_csv('altcoin_dataset_for_DB.csv')
df_fin_asset_prep.to_csv('fin_assets_dataset_for_DB.csv')
df_mining_prep.to_csv('mining_dataset_for_DB.csv')
df_exchanges_prep.to_csv('exchanges_dataset_for_DB.csv')
df_markets_prep.to_csv('markets_dataset_for_DB.csv')
df_dates_prep.to_csv('date_table_for_DB.csv')

### Create DB and load data

In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)
meta = db.MetaData(engine)

with engine.connect() as con:
    con.execute('DROP TABLE IF EXISTS BTC_Data;')
    con.execute('DROP TABLE IF EXISTS Altcoin_Data;')
    con.execute('DROP TABLE IF EXISTS Fin_Assets_Data;')
    con.execute('DROP TABLE IF EXISTS Mining_Data;')
    con.execute('DROP TABLE IF EXISTS Markets;')
    con.execute('DROP TABLE IF EXISTS Exchanges;')
    con.execute('DROP TABLE IF EXISTS Dates_Table;')
    con.close()
        

In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)
meta = db.MetaData(engine)

with engine.connect() as con:
    con.execute('''
        CREATE TABLE IF NOT EXISTS BTC_Data
              (
                ID INT PRIMARY KEY,
                Open REAL NULL DEFAULT 0,
                High REAL NULL DEFAULT 0,
                Low REAL NULL DEFAULT 0,
                Close REAL NULL DEFAULT 0,
                WeightedPrice REAL NULL DEFAULT 0,
                Volume_USD REAL NULL DEFAULT 0,
                Volume_BTC REAL NULL DEFAULT 0,
                Currency VARCHAR(5) NULL DEFAULT "USD",
                MarketID INT NOT NULL,
                ExchangeID INT NOT NULL,
                Date DATE NOT NULL,
                FOREIGN KEY(MarketID) REFERENCES Markets(MarketID) ON DELETE CASCADE,
                FOREIGN KEY(ExchangeID) REFERENCES Exchanges(ExchangeID) ON DELETE CASCADE,
                FOREIGN KEY(Date) REFERENCES Dates_Table(Date) ON DELETE CASCADE
                          
              );''')


In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)
meta = db.MetaData(engine)

with engine.connect() as con:
    con.execute('''
        CREATE TABLE IF NOT EXISTS Altcoin_Data
              (
                ID INT PRIMARY KEY,
                Open REAL NULL DEFAULT 0,
                High REAL NULL DEFAULT 0,
                Low REAL NULL DEFAULT 0,
                Close REAL NULL DEFAULT 0,
                WeightedPrice REAL NULL DEFAULT 0,
                Volume_Market REAL NULL DEFAULT 0,
                Volume_BTC REAL NULL DEFAULT 0,
                MarketID INT NOT NULL,
                ExchangeID INT NOT NULL,
                Date DATE NOT NULL,
                FOREIGN KEY(MarketID) REFERENCES Markets(MarketID) ON DELETE CASCADE,
                FOREIGN KEY(ExchangeID) REFERENCES Exchanges(ExchangeID) ON DELETE CASCADE,
                FOREIGN KEY(Date) REFERENCES Dates_Table(Date) ON DELETE CASCADE
                          
              ); ''')
    con.close()

In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)
meta = db.MetaData(engine)

with engine.connect() as con:
    con.execute('''
        CREATE TABLE IF NOT EXISTS Fin_Assets_Data
              (
                ID INT PRIMARY KEY,
                Open REAL NULL DEFAULT 0,
                High REAL NULL DEFAULT 0,
                Low REAL NULL DEFAULT 0,
                Close REAL NULL DEFAULT 0,
                AdjClose REAL NULL DEFAULT 0,
                Volume REAL NULL DEFAULT 0,
                Asset VARCHAR(50) NULL,
                Symbol VARCHAR(10) NULL,
                Source VARCHAR(10) NULL,
                MarketID INT NOT NULL,
                ExchangeID INT NOT NULL,
                Date DATE NOT NULL,
                FOREIGN KEY(MarketID) REFERENCES Markets(MarketID) ON DELETE CASCADE,
                FOREIGN KEY(ExchangeID) REFERENCES Exchanges(ExchangeID) ON DELETE CASCADE,
                FOREIGN KEY(Date) REFERENCES Dates_Table(Date) ON DELETE CASCADE
                          
              ); ''')
    con.close()

In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)
meta = db.MetaData(engine)

with engine.connect() as con:
    con.execute('''
        CREATE TABLE IF NOT EXISTS Mining_Data
              (
                ID INT PRIMARY KEY,
                Total_BTC INT NULL DEFAULT 0,
                MarketCap INT NULL DEFAULT 0,
                TR_24h REAL NULL DEFAULT 0,
                TR_Avg_h REAL NULL DEFAULT 0,
                BTC_Sent_24h REAL NULL DEFAULT 0,
                BTC_Sent_Avg_h REAL NULL DEFAULT 0,
                BTC_Count INT NULL DEFAULT 0,
                Blocks_24h INT NULL DEFAULT 0,
                Blocks_Avg_h REAL NULL DEFAULT 0,
                Difficulty INT NULL DEFAULT 0,
                Next_Difficulty INT NULL DEFAULT 0,
                Hashrate_Tera REAL NULL DEFAULT 0,
                Hashrate_Peta REAL NULL DEFAULT 0,
                MarketID INT NOT NULL,
                Date DATE NOT NULL,
                FOREIGN KEY(MarketID) REFERENCES Markets(MarketID) ON DELETE CASCADE,
                FOREIGN KEY(Date) REFERENCES Dates_Table(Date) ON DELETE CASCADE
                          
              ); ''')
    con.close()

In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)
meta = db.MetaData(engine)

with engine.connect() as con:
    con.execute('''
        CREATE TABLE IF NOT EXISTS Markets
              (
                MarketID INT PRIMARY KEY,
                Market VARCHAR(50) NOT NULL,
                Symbol VARCHAR(10) NOT NULL,
                MarketName VARCHAR(50) NULL,
                MarketCap INT NULL DEFAULT 0,
                Rank_MarketCap INT NOT NULL,
                Total_Volume INT NULL DEFAULT 0,
                Total_Supply INT NULL DEFAULT 0,
                Max_Supply INT NULL DEFAULT 0
                          
              ); ''')
    con.close()

In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)
meta = db.MetaData(engine)

with engine.connect() as con:
    con.execute('''
        CREATE TABLE IF NOT EXISTS Exchanges
              (
                ExchangeID INT PRIMARY KEY,
                Exchange VARCHAR(50) NOT NULL,
                ExchangeName VARCHAR(50) NULL,
                Year_Est REAL NULL,
                Country VARCHAR(50) NULL,
                Trading_Inc REAL NULL,
                Trust_Score INT NOT NULL,
                Rank_Trust INT NOT NULL,
                Trade_Vol_24h REAL NULL DEFAULT 0,
                Norm_Trade_Vol REAL NULL DEFAULT 0
                          
              ); ''')
    con.close()

In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)
meta = db.MetaData(engine)

with engine.connect() as con:
    con.execute('''
        CREATE TABLE IF NOT EXISTS Dates_Table
              (
                Date DATE PRIMARY KEY,
                Week_day VARCHAR(50) NOT NULL,
                Day INT NOT NULL,
                Month INT NOT NULL,
                Week INT NOT NULL,
                Quarter INT NOT NULL,
                Year INT NOT NULL
                          
              ); ''')
    con.close()

In [ ]:
inspector = db.inspect(engine)
table_names = inspector.get_table_names()
table_names

In [ ]:
for table in table_names:
    load_to_sql(table)

In [ ]:
engine = db.create_engine('sqlite:///crypto.db', echo=True)

with engine.connect() as con:
    df_sql = pd.read_sql_query("""
    SELECT * 
    FROM BTC_DATA;
        """, con)
    con.close()

In [ ]:
df_sql

## Data Wrangling

### BTC Dataset Preparation using Pandas

In [ ]:
btc_data_prepared = df_btc_prep.merge(df_exchanges_prep[['Exchange', 'ExchangeID']], how='left', 
                                      left_on='ExchangeID', right_on='ExchangeID').drop(columns=['ExchangeID'])
btc_data_prepared['Exchange'] = btc_data_prepared['Exchange'].str.upper()
btc_data_prepared['Month'] = btc_data_prepared['Date'].dt.month_name()
btc_data_prepared['Year'] = btc_data_prepared['Date'].dt.year
btc_data_prepared = btc_data_prepared[['Date', 'WeightedPrice', 'Volume_BTC', 'Exchange', 'Year', 'Month']]
btc_data_prepared.tail()

In [ ]:
btc_data_prepared.isna().sum()

In [ ]:
btc_data_prepared.loc[(btc_data_prepared==0).any(axis=1)]

In [ ]:
btc_data_prepared = btc_data_prepared.replace(0, np.nan)

In [ ]:
btc_data_prepared.isna().sum()

In [ ]:
btc_data_prepared[(btc_data_prepared['Date'].dt.year==2014) & (btc_data_prepared['Exchange']=='KRAKEN') 
                  & (btc_data_prepared.index.to_series().between(211, 240))]

In [ ]:
btc_data_prepared['WeightedPrice_filled'] = btc_data_prepared.groupby(['Exchange', 'Year', 'Month'])['WeightedPrice'].transform(
    lambda grp: grp.fillna(np.mean(grp)))

In [ ]:
btc_data_prepared['Volume_filled'] = btc_data_prepared.groupby(['Exchange', 'Year', 'Month'])['Volume_BTC'].transform(
    lambda grp: grp.fillna(np.mean(grp)))

In [ ]:
btc_data_prepared[(btc_data_prepared['Date'].dt.year==2014) & (btc_data_prepared['Exchange']=='KRAKEN') 
                  & (btc_data_prepared.index.to_series().between(211, 240))]

In [ ]:
btc_data_prepared

In [ ]:
btc_data_prepared = btc_data_prepared.groupby(['Date'], as_index=False).agg({'WeightedPrice_filled':'mean', 'Volume_filled': 'sum'})

In [ ]:
btc_data_prepared[btc_data_prepared['Date']=='2021-03-31']

In [ ]:
btc_data_prepared.isna().sum()

In [ ]:
# Simple 30-Day Moving Average

btc_data_prepared['SMA_30'] = btc_data_prepared.loc[:,'WeightedPrice_filled'].rolling(window=30).mean().fillna(method='bfill')

In [ ]:
# Exponential Moving Average

btc_data_prepared['EMA'] = btc_data_prepared.loc[:,'WeightedPrice_filled'].ewm(span=40, adjust=False).mean()

In [ ]:
btc_data_prepared.set_index('Date', inplace=True)

In [ ]:
btc_data_prepared

In [ ]:
fig = go.Figure()

date = btc_data_prepared.index
price = btc_data_prepared['WeightedPrice_filled']
volume = btc_data_prepared['Volume_filled']
ma = btc_data_prepared['SMA_30']
ema = btc_data_prepared['EMA']


fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                   vertical_spacing=0.1, subplot_titles=('BTC Price USD', '# BTC Volume'),
                   row_width=[0.5, 1])

fig.add_trace(go.Scatter(x=date, y=price,
                    mode='lines',
                    name='BTC Weighted Price'), row=1, col=1)

fig.add_trace(go.Scatter(x=date, y=ma,
                    mode='lines',
                    name='BTC 30-Day Moving Avg'), row=1, col=1)

fig.add_trace(go.Scatter(x=date, y=ema,
                    mode='lines',
                    name='BTC Exp Weighted Avg'), row=1, col=1)

fig.add_trace(go.Bar(x=date, y=volume,
                     name='Volume',
                     opacity=0.2,
                     marker=dict(
                         line=dict(color='firebrick', width=2)
                     )), row=2, col=1)

fig.update_annotations(font_size=12)

title = "BTC Price development"
                              
fig.update_layout(title=title,
                  dragmode='select',
                  width=1000,
                  height=600,
                  hovermode='closest',
                  yaxis_type='log'
                 )


fig.show()

In [ ]:
btc_data_prepared.info()

In [ ]:
btc_data_prepared

In [ ]:
df_altcoin_prep.info()

In [ ]:
df_fin_asset_prep.info()

In [ ]:
altcoin_data_prepared = df_altcoin_prep.merge(df_markets_prep[['Symbol', 'MarketID']], how='left', 
                                      left_on='MarketID', right_on='MarketID').drop(columns=['MarketID'])
altcoin_data_prepared['Symbol'] = altcoin_data_prepared['Symbol'].str.upper()
altcoin_data_prepared['Month'] = altcoin_data_prepared['Date'].dt.month_name()
altcoin_data_prepared['Year'] = altcoin_data_prepared['Date'].dt.year
altcoin_data_prepared_copy = altcoin_data_prepared.copy()
altcoin_data_prepared = altcoin_data_prepared[['Date', 'WeightedPrice', 'Symbol', 'Year', 'Month']]
altcoin_data_prepared.tail()

In [ ]:
altcoin_data_prepared.isna().sum()

In [ ]:
altcoin_data_prepared.loc[(altcoin_data_prepared==0).any(axis=1)]

In [ ]:
altcoin_data_prepared[altcoin_data_prepared['Symbol']=='ETH']

In [ ]:
altcoin_data_grouped = altcoin_data_prepared.groupby(['Date'], as_index=False).agg({'WeightedPrice':'mean'})

In [ ]:
altcoin_data_grouped.set_index('Date', inplace=True)

In [ ]:
altcoin_data_grouped = altcoin_data_grouped.merge(btc_data_prepared['WeightedPrice_filled'], how='left',
                                                   left_index=True, right_index=True)

In [ ]:
altcoin_data_grouped['WeightedPrice_USD'] = altcoin_data_grouped['WeightedPrice'] * altcoin_data_grouped['WeightedPrice_filled']

In [ ]:
altcoin_data_grouped

In [ ]:
altcoin_data_grouped.drop(columns=['WeightedPrice', 'WeightedPrice_filled'], inplace=True)

In [ ]:
# Exponential Moving Average

altcoin_data_grouped['Altcoin_EMA'] = altcoin_data_grouped.loc[:,'WeightedPrice_USD'].ewm(span=40, adjust=False).mean()

In [ ]:
altcoin_data_grouped

In [ ]:
btc_altcoin_merged = btc_data_prepared.merge(altcoin_data_grouped['Altcoin_EMA'], how='left', left_index=True, right_index=True)

In [ ]:
btc_altcoin_merged.isna().sum()

In [ ]:
btc_altcoin_merged[btc_altcoin_merged['Altcoin_EMA'].isna()]

In [ ]:
btc_altcoin_merged['Altcoin_EMA_filled'] = btc_altcoin_merged['Altcoin_EMA'].fillna(
    value=btc_altcoin_merged['Altcoin_EMA'].rolling(window=30, center=True, min_periods=2).mean())

In [ ]:
btc_altcoin_merged.isna().sum()

In [ ]:
btc_altcoin_merged.head(850)

In [ ]:
btc_altcoin_prepared = btc_altcoin_merged[btc_altcoin_merged.index>='2014-01-05']

In [ ]:
btc_altcoin_prepared.drop('Altcoin_EMA', axis=1, inplace=True)

In [ ]:
btc_altcoin_prepared.rename(columns={'WeightedPrice_filled': 'WeightedPrice', 'Volume_filled': 'Volume', 
                                     'EMA': 'EMA_40', 'Altcoin_EMA_filled': 'Altcoin_EMA_40'}, inplace=True)

In [ ]:
btc_altcoin_prepared.isna().sum()

In [ ]:
btc_altcoin_prepared

In [ ]:
altcoin_data_prepared.tail()

In [ ]:
altcoin_data_prepared.info()

In [ ]:
altcoin_data_prepared.loc[(altcoin_data_prepared==0).any(axis=1)].count()

In [ ]:
altcoin_data_prepared_copy.tail()

In [ ]:
altcoin_data_prepared_copy.info()

In [ ]:
altcoin_for_top10 = altcoin_data_prepared.merge(altcoin_data_prepared_copy[['Date', 'Symbol', 'Volume_BTC']], left_on=['Date', 'Symbol'], right_on=['Date', 'Symbol'])

In [ ]:
altcoin_for_top10

In [ ]:
top10_altcoin = pd.DataFrame(altcoin_for_top10.groupby('Symbol')['Volume_BTC'].sum().nlargest(10))

In [ ]:
top10 = top10_altcoin.reset_index()['Symbol']

In [ ]:
top10

In [ ]:
altcoin_data_to_merge = altcoin_data_prepared.loc[altcoin_data_prepared['Symbol'].isin(top10)]

In [ ]:
altcoin_data_to_merge.set_index('Date', inplace=True)

In [ ]:
altcoin_data_to_merge

In [ ]:
altcoin_data_to_merge = altcoin_data_to_merge.merge(btc_data_prepared['WeightedPrice_filled'], how='left', left_index=True, right_index=True)

In [ ]:
altcoin_data_to_merge

In [ ]:
altcoin_data_to_merge['WeightedPrice_USD'] = altcoin_data_to_merge['WeightedPrice'] * altcoin_data_to_merge['WeightedPrice_filled']

In [ ]:
altcoin_data_to_pivot = altcoin_data_to_merge[['WeightedPrice_USD', 'Symbol']]

In [ ]:
altcoin_data_to_pivot

In [ ]:
altcoin_data_to_pivot.isna().sum()

In [ ]:
altcoin_data_pivoted = altcoin_data_to_pivot.pivot_table('WeightedPrice_USD', ['Date'], 'Symbol')

In [ ]:
altcoin_data_pivoted

In [ ]:
altcoin_data_pivoted['Year'] = altcoin_data_pivoted.index.year
altcoin_data_pivoted['Month'] = altcoin_data_pivoted.index.month_name()
altcoin_data_pivoted

In [ ]:
altcoin_data_pivoted.isna().sum()

In [ ]:
for col in altcoin_data_pivoted.columns[:10]:
    print(str(col))

In [ ]:
altcoin_data_pivoted.groupby(['Year', 'Month'])['DASH'].mean()

In [ ]:
altcoin_data_pivoted.iloc[0]['DASH']

In [ ]:
list(set(altcoin_data_pivoted['Year']))

In [ ]:
# Fill the NaN with the mean price per year & month

years = list(set(altcoin_data_pivoted['Year']))
months = list(set(altcoin_data_pivoted['Month']))

for year in years:
    for month in months:
        for col in altcoin_data_pivoted.columns[:10]:
            altcoin_data_pivoted[col].fillna(altcoin_data_pivoted[col][(altcoin_data_pivoted['Year']==year) & (altcoin_data_pivoted['Month']==month)].mean(), inplace=True)

In [ ]:
altcoin_data_pivoted

In [ ]:
altcoin_data_pivoted.isna().sum()

In [ ]:
btc_dataset_merged = btc_altcoin_prepared.merge(altcoin_data_pivoted, how='left', left_index=True, right_index=True)

In [ ]:
btc_altcoin_dataset_final = btc_dataset_merged.drop(['Year', 'Month'], axis=1)
btc_altcoin_dataset_final['Day_of_Week'] = btc_altcoin_dataset_final.index.day_name()

In [ ]:
btc_altcoin_dataset_final

In [ ]:
btc_altcoin_dataset_final.isna().sum()

In [ ]:
btc_altcoin_dataset_final.loc[(btc_altcoin_dataset_final==0).any(axis=1)]

In [ ]:
btc_altcoin_dataset_cleansed = btc_altcoin_dataset_final.dropna()

In [ ]:
btc_altcoin_dataset_cleansed.isna().sum()

In [ ]:
btc_altcoin_dataset_cleansed.tail()

In [ ]:
btc_altcoin_dataset_cleansed.to_csv('btc_altcoin_dataset_cleansed.csv')

In [ ]:
df_check = pd.read_csv('btc_altcoin_dataset_cleansed.csv')
df_check.tail()

## EDA on the prepared dataset

### Pairplots

In [ ]:
sns.pairplot(btc_altcoin_dataset_cleansed, diag_kind='kde', corner=True)

### Hypothesis Testing

In [ ]:
btc_altcoin_OLS = btc_altcoin_prepared[['WeightedPrice', 'Altcoin_EMA_40']].reset_index().drop('Date', axis=1)

In [ ]:
btc_altcoin_OLS

In [ ]:
btc_altcoin_OLS.describe()

In [ ]:
btc_altcoin_OLS_pc = btc_altcoin_OLS.pct_change()

In [ ]:
np.isfinite(btc_altcoin_OLS_pc)

In [ ]:
btc_altcoin_OLS_pc.isna().sum()

In [ ]:
btc_altcoin_OLS_pc.dropna(inplace=True)

In [ ]:
qqplot(btc_altcoin_OLS_pc['WeightedPrice'], line='s')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(btc_altcoin_OLS_pc['Altcoin_EMA_40'],
                                                    btc_altcoin_OLS_pc['WeightedPrice'],
                                                    test_size=0.2, random_state=12)

In [ ]:
X = btc_altcoin_OLS['Altcoin_EMA_40']
X = sm.add_constant(X)
Y = btc_altcoin_OLS['WeightedPrice']

In [ ]:
model = sm.OLS(Y, X).fit()
predictions = model.predict(X)

In [ ]:
print_model = model.summary()
print(print_model)

#### Interpretation

<ul>
    <li>Adj R-square: good model fit</li>
    <li>Constant: Independent Average price of BTC over 7 Years</li>
    <li>Std error: high level of accuracy</li>
    <li>p-value: Model is statistically significant</li>
    <li>Durbin-Watson test: a positive autocorrelation is present, eg a EMA of the Altcoins yesterday is indicating a greater likelihood in BTC price increase in the future</li>
    
    
</ul>